# 深度学习计算
---
## 5.1 层和块

总结：

1. 一个块可以由许多层组成；一个块可以由许多块组成。

2. 块可以包含代码。

3. 块负责大量的内部处理，包括参数初始化和反向传播。

4. 层和块的顺序连接由Sequential块处理。

### 多层感知机

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.randn(2, 20)
net(X)

tensor([[-0.2610,  0.3517, -0.1073,  0.0457,  0.1955,  0.0990,  0.2351,  0.2947,
          0.3182, -0.3104],
        [-0.3461, -0.0939,  0.0275, -0.0470,  0.2726,  0.2285, -0.2116,  0.3026,
         -0.2395, -0.2627]], grad_fn=<AddmmBackward0>)

### 自定义块

1. 将输入数据作为其前向传播函数的参数。

2. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

4. 存储和访问前向传播计算所需的参数。

5. 根据需要初始化模型参数。

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，声明两个全连接层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) #隐藏层
        self.out = nn.Linear(256,10)# 输出层
        
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))
    
net = MLP()
net(X)

tensor([[ 0.5009,  0.2235,  0.3930,  0.1576, -0.2799, -0.5207,  0.1694, -0.0191,
          0.2811, -0.4751],
        [ 0.3258, -0.1251,  0.0582,  0.0028, -0.2791, -0.4827, -0.0259,  0.0642,
          0.4901, -0.2563]], grad_fn=<AddmmBackward0>)

### 顺序块

In [11]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
         # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
        
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[-0.1634,  0.7588, -0.1810,  0.0728, -0.2036, -0.0607, -0.1814, -0.2863,
          0.2656, -0.1135],
        [-0.1611,  0.1043,  0.0776,  0.6293,  0.3566, -0.0177, -0.2202, -0.0684,
         -0.0380,  0.0264]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        #使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        #复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 20
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.1055, grad_fn=<SumBackward0>)

### 嵌套块  （无限套娃）

In [14]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1840, grad_fn=<SumBackward0>)